In [1]:
# Essentials
import numpy as np
import pandas as pd
import datetime
import random
import itertools
# Plots
import seaborn as sns
import matplotlib.pyplot as plt

import h2o
from h2o.automl import H2OAutoML

In [2]:
train = pd.read_csv('../input/widsdatathon2022/train.csv')
test = pd.read_csv('../input/widsdatathon2022/test.csv')
cols1=['State_Factor','building_class','facility_type','floor_area','year_built']
train['source'] = 'train'
test['source']  = 'test'
df = pd.concat([train,test],0,ignore_index=True)
df=df.sort_values(by=cols1+['Year_Factor'])

# agregate by meteo station so same location 
min_temp = [col for col in df.columns if 'min_temp' in col]
max_temp = [col for col in df.columns if 'max_temp' in col]
avg_temp = [col for col in df.columns if 'avg_temp' in col]

df['min_temp1'] = df[min_temp].min(axis=1)
df['max_temp1'] = df[max_temp].min(axis=1)
df['avg_temp1'] = df[avg_temp].min(axis=1)

df['min_temp2'] = df[min_temp].max(axis=1)
df['max_temp2'] = df[max_temp].max(axis=1)
df['avg_temp2'] = df[avg_temp].max(axis=1)

df['min_temp3'] = df[min_temp].mean(axis=1)
df['max_temp3'] = df[max_temp].mean(axis=1)
df['avg_temp3'] = df[avg_temp].mean(axis=1)

df['min_temp4'] = df[min_temp].std(axis=1)
df['max_temp4'] = df[max_temp].std(axis=1)
df['avg_temp4'] = df[avg_temp].std(axis=1)

df['min_temp5'] = df[min_temp].skew(axis=1)
df['max_temp5'] = df[max_temp].skew(axis=1)
df['avg_temp5'] = df[avg_temp].skew(axis=1)


cols1=['State_Factor','building_class','facility_type','floor_area','year_built']


# lag and delta from current values
for i in [1,2,3] :
    df[f'site_eui_lag{i}']          =df.groupby(cols1)['site_eui'].shift(i)
    df[f'Year_Factor_lag{i}']       =df.groupby(cols1)['Year_Factor'].shift(i)
    df[f'energy_star_rating_lag{i}']=df.groupby(cols1)['energy_star_rating'].shift(i)
    df[f'ELEVATION_lag{i}']=df.groupby(cols1)['ELEVATION'].shift(i)
    
    df['ELEVATION_FLOOR'] = df['ELEVATION'] * df['floor_area']
    df[f'ELEVATION_FLOOR_lag{i}'] = df[f'ELEVATION_lag{i}'] * df['floor_area']
    
    df['ELEVATION_FLOORB'] = df['ELEVATION'] / df['floor_area']
    df[f'ELEVATION_FLOORB_lag{i}'] = df[f'ELEVATION_lag{i}'] / df['floor_area']
    
    
    df[f'delta_energy{i}']  = np.where( df[f'energy_star_rating'].notnull() & df[f'energy_star_rating_lag{i}'].notnull() ,
                                       (df['energy_star_rating']-df[f'energy_star_rating_lag{i}'] ) / df['energy_star_rating'],
                                        np.nan )
    
    df[f'delta_energy{i}B'] = np.where( df[f'energy_star_rating'].notnull() & df[f'energy_star_rating_lag{i}'].notnull() ,
                                       (df['energy_star_rating']-df[f'energy_star_rating_lag{i}'] ) ,
                                      np.nan )
    
    
    df[f'site_eui{i}B'] = np.where( df[f'site_eui_lag{i}'].notnull() & df[f'delta_energy{i}B'].notnull() ,
                                   df[f'site_eui_lag{i}'] * df[f'delta_energy{i}B']  ,
                                   np.nan )
    
    df[f'site_eui{i}C']      = np.where( df[f'site_eui_lag{i}'].notnull() & df[f'site_eui{i}B'].notnull() ,
                                   df[f'site_eui_lag{i}'] - df[f'site_eui{i}B'] ,
                                   np.nan )
    
    df[f'delta_ELEVATION']   = np.where( df[f'ELEVATION'].notnull() & df[f'ELEVATION_lag{i}'].notnull() ,
                                      (df['ELEVATION']-df[f'ELEVATION_lag{i}'] ) ,
                                      np.nan )
    
    df[f'delta_Year_Factor'] = np.where( df[f'Year_Factor'].notnull() & df[f'Year_Factor_lag{i}'].notnull() ,
                                        (df['Year_Factor']-df[f'Year_Factor_lag{i}'] ) ,
                                        np.nan )

# lag and delta from current values

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:6: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  


In [3]:
if 1 :
    i=1
    j=2
    print(i,j)    
    
    df[f'delta_energy{i}_{j}']  = np.where( df[f'energy_star_rating_lag{i}'].notnull() & df[f'energy_star_rating_lag{j}'].notnull() ,
                                           (df[f'energy_star_rating_lag{i}']-df[f'energy_star_rating_lag{j}'] ) / df[f'energy_star_rating_lag{i}'],
                                        np.nan )
    
    df[f'delta_energy{i}_{j}B'] = np.where( df[f'energy_star_rating_lag{i}'].notnull() & df[f'energy_star_rating_lag{j}'].notnull() ,
                                          ( df[f'energy_star_rating_lag{i}']-df[f'energy_star_rating_lag{j}'] ) ,
                                      np.nan )
    
    
    df[f'site_eui{i}_{j}B'] = np.where( df[f'site_eui_lag{i}'].notnull() & df[f'delta_energy{i}_{j}B'].notnull() ,
                                   df[f'site_eui_lag{i}'] * df[f'delta_energy{i}_{j}B']  ,
                                   np.nan )
    
    df[f'site_eui{i}_{j}C']      = np.where( df[f'site_eui_lag{i}'].notnull() & df[f'site_eui{i}_{j}B'].notnull() ,
                                   df[f'site_eui_lag{i}'] - df[f'site_eui{i}_{j}B'] ,
                                   np.nan )
    
    df[f'delta_ELEVATION{i}_{j}']   = np.where( df[f'ELEVATION_lag{j}'].notnull() & df[f'ELEVATION_lag{i}'].notnull() ,
                                      (df[f'ELEVATION_lag{i}']-df[f'ELEVATION_lag{j}'] ) ,
                                      np.nan )
    
    df[f'delta_Year_Factor{i}_{j}'] = np.where( df[f'Year_Factor_lag{j}'].notnull() & df[f'Year_Factor_lag{i}'].notnull() ,
                                        (df[f'Year_Factor_lag{i}']-df[f'Year_Factor_lag{j}'] ) ,
                                        np.nan )

1 2


In [4]:
pd.set_option('display.max_columns', None)
df.head()

,Year_Factor,State_Factor,building_class,facility_type,floor_area,year_built,energy_star_rating,ELEVATION,january_min_temp,january_avg_temp,january_max_temp,february_min_temp,february_avg_temp,february_max_temp,march_min_temp,march_avg_temp,march_max_temp,april_min_temp,april_avg_temp,april_max_temp,may_min_temp,may_avg_temp,may_max_temp,june_min_temp,june_avg_temp,june_max_temp,july_min_temp,july_avg_temp,july_max_temp,august_min_temp,august_avg_temp,august_max_temp,september_min_temp,september_avg_temp,september_max_temp,october_min_temp,october_avg_temp,october_max_temp,november_min_temp,november_avg_temp,november_max_temp,december_min_temp,december_avg_temp,december_max_temp,cooling_degree_days,heating_degree_days,precipitation_inches,snowfall_inches,snowdepth_inches,avg_temp,days_below_30F,days_below_20F,days_below_10F,days_below_0F,days_above_80F,days_above_90F,days_above_100F,days_above_110F,direction_max_wind_speed,direction_peak_wind_speed,max_wind_speed,days_with_fog,site_eui,id,source,min_temp1,max_temp1,avg_temp1,min_temp2,max_temp2,avg_temp2,min_temp3,max_temp3,avg_temp3,min_temp4,max_temp4,avg_temp4,min_temp5,max_temp5,avg_temp5,site_eui_lag1,Year_Factor_lag1,energy_star_rating_lag1,ELEVATION_lag1,ELEVATION_FLOOR,ELEVATION_FLOOR_lag1,ELEVATION_FLOORB,ELEVATION_FLOORB_lag1,delta_energy1,delta_energy1B,site_eui1B,site_eui1C,delta_ELEVATION,delta_Year_Factor,site_eui_lag2,Year_Factor_lag2,energy_star_rating_lag2,ELEVATION_lag2,ELEVATION_FLOOR_lag2,ELEVATION_FLOORB_lag2,delta_energy2,delta_energy2B,site_eui2B,site_eui2C,site_eui_lag3,Year_Factor_lag3,energy_star_rating_lag3,ELEVATION_lag3,ELEVATION_FLOOR_lag3,ELEVATION_FLOORB_lag3,delta_energy3,delta_energy3B,site_eui3B,site_eui3C,delta_energy1_2,delta_energy1_2B,site_eui1_2B,site_eui1_2C,delta_ELEVATION1_2,delta_Year_Factor1_2
1456,3,State_1,Commercial,Commercial_Other,10149.0,1931.0,NaN,45.7,26,44.516129,64,30,49.392857,69,37,55.967742,77,40,61.783333,90,45,65.129032,91,52,70.016667,104,52,71.387097,102,54,71.322581,97,47,69.133333,98,43,61.000000,84,36,55.116667,77,24,45.387097,68,826,2542,4.50,0.0,0,60.056164,20,0,0,0,111,31,5,0,NaN,NaN,NaN,NaN,6.800000,1456,train,24,64,44.516129,54,104,71.387097,40.500000,85.083333,60.016054,10.184659,13.944424,9.478089,-0.304526,-0.152554,-0.395977,NaN,NaN,NaN,NaN,463809.3,NaN,0.004503,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2673,4,State_1,Commercial,Commercial_Other,10149.0,1931.0,NaN,120.4,41,55.096774,73,42,54.875000,71,47,59.032258,76,47,60.100000,90,51,63.483871,92,52,64.250000,87,53,66.903226,90,57,67.016129,80,56,68.383333,83,55,68.661290,95,46,59.650000,76,41,56.790323,68,497,1477,20.69,0.0,0,62.067123,0,0,0,0,27,6,0,0,NaN,NaN,NaN,NaN,8.200000,2673,train,41,68,54.875000,57,95,68.661290,49.000000,81.750000,62.023794,5.815340,9.026476,4.874126,-0.133130,-0.033319,-0.071409,6.8,3.0,NaN,45.7,1221939.6,463809.3,0.011863,0.004503,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3769,5,State_1,Commercial,Commercial_Other,10149.0,1931.0,NaN,59.1,29,51.387097,77,36,56.803571,75,40,61.435484,85,41,60.966667,91,45,62.887097,90,55,72.983333,106,60,74.516129,103,58,74.774194,104,53,73.583333,105,49,69.580645,96,29,52.866667,75,28,48.580645,68,1319,1823,7.95,0.0,0,63.398630,4,0,0,0,136,52,10,0,NaN,NaN,NaN,NaN,12.300000,3769,train,28,68,48.580645,60,106,74.774194,43.583333,89.583333,63.366422,11.571582,13.514022,9.197361,-0.040905,-0.202397,-0.170424,8.2,4.0,NaN,120.4,599805.9,1221939.6,0.005823,0.011863,NaN,NaN,NaN,NaN,NaN,NaN,6.8,3.0,NaN,45.7,463809.3,0.004503,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,74.7,1.0
76077,7,State_1,Commercial,Commercial_Other,10149.0,1931.0,NaN,59.1,38,50.596774,64,40,54.482143,66,42,56.935484,77,45,58.450000,78,49,60.903226,87,51,63.150000,97,53,64.258065,85,53,65.854839,88,54,69.766667,104,48,64.193548,91,

In [5]:
df.year_built.max()

2016.0

In [6]:
grp1 = df.groupby(min_temp,as_index=True).agg({
            "floor_area": ['mean', 'max','min','sum']  ,
            "year_built": ['mean', 'median','max','min'] ,
            "energy_star_rating" : ['min','max','mean', 'median'] ,
            "id" : "count"
            })
grp1.columns = ["_".join(x) for x in grp1.columns.ravel()  ]
print(df.shape)
df = df.merge(grp1,on=min_temp,how='left')
print(df.shape)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:7: FutureWarning: Index.ravel returning ndarray is deprecated; in a future version this will return a view on self.
  import sys


(85462, 120)
(85462, 133)


In [7]:
df = df.sort_values(by='id')


Prepare for Training :

In [8]:
train = df.loc[df['source']=='train']
print(train.shape)
train.to_csv('train_h2O.csv',index=False)
df.loc[df['source']=='test'].to_csv('test_h2O.csv',index=False)

(75757, 133)


In [9]:
train.Year_Factor.value_counts()

6    22449
5    18308
4    12946
3    10879
2     9058
1     2117
Name: Year_Factor, dtype: int64

In [10]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.11" 2021-04-20; OpenJDK Runtime Environment (build 11.0.11+9-Ubuntu-0ubuntu2.20.04); OpenJDK 64-Bit Server VM (build 11.0.11+9-Ubuntu-0ubuntu2.20.04, mixed mode, sharing)
  Starting server from /opt/conda/lib/python3.7/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpp8q4us55
  JVM stdout: /tmp/tmpp8q4us55/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmpp8q4us55/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,02 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.34.0.3
H2O_cluster_version_age:,4 months and 21 days !!!
H2O_cluster_name:,H2O_from_python_unknownUser_83r4or
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,4 Gb
H2O_cluster_total_cores:,4
H2O_cluster_allowed_cores:,4
H2O_cluster_status:,"locked, healthy"


In [11]:
train =  h2o.import_file('train_h2O.csv')
test =  h2o.import_file('test_h2O.csv')

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [12]:
x = train.columns
y = 'site_eui'
x.remove(y)
x.remove("id") #removing id  
x = [ c for c in x if 'Year_Factor_lag' not in c ]
x.remove('Year_Factor')
x.remove('source')
x

['State_Factor',
 'building_class',
 'facility_type',
 'floor_area',
 'year_built',
 'energy_star_rating',
 'ELEVATION',
 'january_min_temp',
 'january_avg_temp',
 'january_max_temp',
 'february_min_temp',
 'february_avg_temp',
 'february_max_temp',
 'march_min_temp',
 'march_avg_temp',
 'march_max_temp',
 'april_min_temp',
 'april_avg_temp',
 'april_max_temp',
 'may_min_temp',
 'may_avg_temp',
 'may_max_temp',
 'june_min_temp',
 'june_avg_temp',
 'june_max_temp',
 'july_min_temp',
 'july_avg_temp',
 'july_max_temp',
 'august_min_temp',
 'august_avg_temp',
 'august_max_temp',
 'september_min_temp',
 'september_avg_temp',
 'september_max_temp',
 'october_min_temp',
 'october_avg_temp',
 'october_max_temp',
 'november_min_temp',
 'november_avg_temp',
 'november_max_temp',
 'december_min_temp',
 'december_avg_temp',
 'december_max_temp',
 'cooling_degree_days',
 'heating_degree_days',
 'precipitation_inches',
 'snowfall_inches',
 'snowdepth_inches',
 'avg_temp',
 'days_below_30F',
 'days_

In [13]:
aml = H2OAutoML(max_runtime_secs= 5555, max_models = 10,
                seed=1,stopping_metric='RMSE',
                include_algos = [ "GLM", "DeepLearning", "DRF","XGBoost","GBM","StackedEnsemble"])

aml.train(x=x, y=y, training_frame=train  , fold_column="Year_Factor")

AutoML progress: |
07:18:36.496: Fold column Year_Factor will be used for cross-validation. nfolds parameter will be ignored.

███████████████████████████████████████████████████████████████| (done) 100%
Model Details
H2OStackedEnsembleEstimator :  Stacked Ensemble
Model Key:  StackedEnsemble_AllModels_1_AutoML_1_20220301_71836

No model summary for this model

ModelMetricsRegressionGLM: stackedensemble
** Reported on train data. **

MSE: 868.7097423624633
RMSE: 29.473882376817333
MAE: 13.119029033219938
RMSLE: 0.32269319309990246
R^2: 0.7493646931733179
Mean Residual Deviance: 868.7097423624633
Null degrees of freedom: 9913
Residual degrees of freedom: 9908
Null deviance: 34362637.00001465
Residual deviance: 8612388.385781461
AIC: 95236.84101834572

ModelMetricsRegressionGLM: stackedensemble
** Reported on cross-validation data. **

MSE: 1316.4671602584951
RMSE: 36.283152567803356
MAE: 16.445712874863798
RMSLE: NaN
R^2: 0.6120789507345796
Mean Residual Deviance: 1316.4671602584951
Nul

In [14]:
# Get AutoML event log
log = aml.event_log

# Get training timing info
info = aml.training_info

In [15]:
log

timestamp,level,stage,message,name,value
07:18:36.494,Info,Workflow,Project: AutoML_1_20220301_71836,,
07:18:36.496,Warn,Validation,Fold column Year_Factor will be used for cross-validation. nfolds parameter will be ignored.,,
07:18:36.507,Info,Validation,Setting stopping tolerance adaptively based on the training frame: 0.0036331942310812997,,
07:18:36.507,Info,Validation,Build control seed: 1,,
07:18:36.509,Info,DataImport,training frame: Frame key: AutoML_1_20220301_71836_training_train_h2O.hex cols: 133 rows: 75757 chunks: 16 size: 18496517 checksum: -3196747993909563614,,
07:18:36.509,Info,DataImport,validation frame: NULL,,
07:18:36.509,Info,DataImport,leaderboard frame: NULL,,
07:18:36.509,Info,DataImport,blending frame: NULL,,
07:18:36.510,Info,DataImport,response column: site_eui,,
07:18:36.510,Info,DataImport,"fold column: [75757,1.0/4.367754794936437/6.0, 49.4 KB, {/127.0.0.1:54321:0:/127.0.0.1:54321:4855:/127.0.0.1:54321:9759:/127.0.0.1:54321:14765:/127.0.0.1:54321:20034:/127.0.0.1:54321:25333:/127.0.0.1:54321:30324:/127.0.0.1:54321:35312:/127.0.0.1:54321:39944:/127.0.0.1:54321:44537:/127.0.0.1:54321:48759:/127.0.0.1:54321:53087:/127.0.0.1:54321:57350:/127.0.0.1:54321:61419:/127.0.0.1:54321:65609:/127.0.0.1:54321:70636:}]",,


In [16]:
info

{'creation_epoch': '1646119116',
 'start_epoch': '1646119117',
 'start_XGBoost_def_2': '1646119117',
 'start_GLM_def_1': '1646119297',
 'start_GBM_def_5': '1646119371',
 'start_StackedEnsemble_best_of_family_1': '1646119563',
 'start_XGBoost_def_1': '1646119565',
 'start_DRF_def_1': '1646119678',
 'start_GBM_def_2': '1646120537',
 'start_GBM_def_3': '1646120660',
 'start_GBM_def_4': '1646120775',
 'start_StackedEnsemble_best_of_family_2': '1646120918',
 'start_StackedEnsemble_all_2': '1646120920',
 'start_XGBoost_def_3': '1646120922',
 'start_DRF_XRT': '1646120997',
 'start_StackedEnsemble_best_of_family_3': '1646121747',
 'start_StackedEnsemble_all_3': '1646121749',
 'start_StackedEnsemble_best_of_family_xgboost': '1646121751',
 'start_StackedEnsemble_best_of_family_gbm': '1646121776',
 'start_StackedEnsemble_all_xgboost': '1646121789',
 'start_StackedEnsemble_all_gbm': '1646121820',
 'start_StackedEnsemble_best_of_family_xglm': '1646121837',
 'start_StackedEnsemble_all_xglm': '164612

In [17]:
lb = aml.leaderboard
lb

model_id,mean_residual_deviance,rmse,mse,mae,rmsle
StackedEnsemble_AllModels_1_AutoML_1_20220301_71836,1316.47,36.2832,1316.47,16.4457,nan
StackedEnsemble_AllModels_2_AutoML_1_20220301_71836,1316.7,36.2864,1316.7,16.4548,nan
StackedEnsemble_AllModels_5_AutoML_1_20220301_71836,1318.28,36.3081,1318.28,16.4759,0.384093
StackedEnsemble_BestOfFamily_3_AutoML_1_20220301_71836,1327.4,36.4335,1327.4,16.642,nan
StackedEnsemble_BestOfFamily_6_AutoML_1_20220301_71836,1328.02,36.4421,1328.02,16.6719,0.388718
GBM_2_AutoML_1_20220301_71836,1328.34,36.4464,1328.34,16.5786,nan
StackedEnsemble_BestOfFamily_2_AutoML_1_20220301_71836,1328.53,36.449,1328.53,16.6479,nan
GBM_4_AutoML_1_20220301_71836,1329.5,36.4623,1329.5,16.8094,nan
GBM_1_AutoML_1_20220301_71836,1358.51,36.858,1358.51,16.629,nan
GBM_3_AutoML_1_20220301_71836,1362.5,36.9121,1362.5,17.2274,nan


In [18]:
preds = aml.predict(test)

stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%


In [19]:
## create submission
sample_submission=pd.read_csv('/kaggle/input/widsdatathon2022/sample_solution.csv')
sample_submission['site_eui'] = h2o.as_list(preds)['predict']
sample_submission.to_csv("submission_automl1.csv",index=None)
sample_submission.head()

,id,site_eui
0,75757,163.202741
1,75758,258.325591
2,75759,173.828553
3,75760,262.820162
4,75761,247.748933
